In [ ]:
import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from Classes.Communicator import Communicator

diya_name = "06"
comm = Communicator(diya_name, verbose=True)

In [ ]:
x_HP = 0
x_FAN = 0

try:
    comm.start()
    comm.send_control_input(x_HP, x_FAN)

except KeyboardInterrupt:
    comm.stop()

In [11]:
import paho.mqtt.client as mqtt

# Define the MQTT broker details
broker = "mqtt.119.ovh"
port = 1883
topic = "diya06"
read_topic = topic + "/rx/"
transmit_topic = topic + "/tx/"
keep_alive = 60

last_message = None

# Callback function when a message is received
def on_message(client, userdata, message):
    last_message = message.payload.decode()
    # print(f"Message received: {last_message} on topic {message.topic}")

    parts = last_message.split()
    if "TMP&U" not in last_message or "Peltier:" not in last_message or "Fan:" not in last_message:
        print("Received incomplete message")
        return
    
    # Parse each component based on its position in the message
    timestamp = parts[0]
    sensor_label = parts[1]
    T_1 = float(parts[2])
    T_2 = float(parts[3])
    T_3 = float(parts[4])

    # Parse the 'Peltier' and 'Fan' values by removing their labels
    peltier_value = int(parts[6])
    fan_value = int(parts[8])

    # Print out parsed values
    print("Timestamp:", timestamp)
    print("Measurements:", T_1, T_2, T_3)
    print("Peltier Value:", peltier_value)
    print("Fan Value:", fan_value)
    print("-------------------------------------------")

def send_control_input(client:mqtt.Client, x_HP:int, x_FAN:int):
    sign_HP = "+" if x_HP >= 0 else "-"

    # Bound control values
    x_HP = max(min(x_HP, 100), -100)
    x_FAN = max(min(x_FAN, 100), 0)

    # Format control values
    x_HP_formatted = f"{sign_HP}{abs(int(x_HP)):03d}"
    x_FAN_formatted = f"{int(x_FAN):03d}"

    message_str = f"S{x_HP_formatted}{x_FAN_formatted}"

    client.publish(transmit_topic, message_str)
    print(f"Message sent: x_HP = {x_HP}, x_FAN = {x_FAN}")


# Create an MQTT client instance
# cself.client = mqtt.Client(api_version=mqtt.CallbackAPIVersion.VERSION2) ##################################3
client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)

# Assign the on_message callback
client.on_message = on_message

# Connect to the broker
client.connect(broker, port, keep_alive)

# Subscribe to the specified topic
client.subscribe(read_topic)

(<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>, 1)

In [ ]:
x_HP = 0
x_FAN = 0

send_control_input(client, x_HP, x_FAN)

client.loop_start()

Message sent: x_HP = 0, x_FAN = 0


<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>

Timestamp: 20000101T192412
Measurements: 23.99 24.09 25.39
Peltier Value: 0
Fan Value: 0
-------------------------------------------
Received incomplete message
Received incomplete message
Received incomplete message
Received incomplete message
Received incomplete message
Timestamp: 20000101T192413
Measurements: 24.01 24.11 25.43
Peltier Value: 0
Fan Value: 0
-------------------------------------------
Received incomplete message
Received incomplete message
Received incomplete message
Received incomplete message
Received incomplete message
Received incomplete message
Received incomplete message
Received incomplete message
Received incomplete message
Received incomplete message
Received incomplete message
Received incomplete message
Timestamp: 20000101T192414
Measurements: 24.03 24.13 25.47
Peltier Value: 0
Fan Value: 0
-------------------------------------------
Received incomplete message
Received incomplete message
Received incomplete message
Received incomplete message
Received inco

In [13]:
client.loop_stop()

<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>

In [9]:
client.disconnect()

<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>